In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
class BabyNames(object):
    def __init__(self, path): 
        newlist = []
        columns = ['state','sex','year', 'name', 'occurances']

        for filename in os.listdir(path):
            if filename.endswith('.TXT'):
                df = pd.read_csv(join(path, filename), names=columns)
                newlist.append(df)
        self.names = pd.concat(newlist,ignore_index = True)
    
    def Count(self, state = '', year = ''): 
        self.state = state
        self.year = year
        if self.state == '': 
            self.state = self.data.state
        if self.year == '':
            self.year = self.data.year
        return(self.data[(self.data.state.isin([self.state])) & (self.data.year.isin([self.year]))]['occurances'].sum())
        
    
    def Top10BabyNames(self, state ='', year = ''):
        if state == '': 
            state = self.data.state
        if year == '':
            year = self.data.year
        t = self.data[(self.data.state.isin([state])) & (self.data.year.isin([year]))]
                                                       
        data = {'Male': t[t.sex == "M"].groupby('name').sum().sort_values(by = 'occurances', ascending = False).head(10).index.values,
                'Female': t[t.sex == "F"].groupby('name').sum().sort_values(by = 'occurances', ascending = False).head(10).index.values
            }
        df = pd.DataFrame(data, index = range(1,11))
        df.index.name = "Rank"
        return(df)
        
        
        
    #def ChangeOfPopularity(fromYear=2014, toYear=2015, top=10):
    #def Top5NamesPerYear(year=2015,sex=’’):
    #def NamePopularityPlot (name=’Jim’, yearRange=(2000,2015), state=’IL’, sex=’M’):
    #def NameFlip(n=10):
        



In [41]:
mypath = './data/namesbystate/'
test = BabyNames(mypath)

In [42]:
test.Count("MI", 2004)

104214

In [44]:
test.Top10BabyNames()

,Female,Male
Rank,,
1,Mary,James
2,Patricia,John
3,Elizabeth,Robert
4,Jennifer,Michael
5,Linda,William
6,Barbara,David
7,Margaret,Richard
8,Susan,Joseph
9,Dorothy,Charles


In [ ]:
        #if state is None and year is None: # no input, returns general sum
        #    return(self.data['occurances'].sum())
        #elif state is not None and year is None: # no year, yes state, returns sum by state
        #    return(self.data[self.data.state == state]['occurances'].sum()) 
        #elif state is None and year is not None: # no state, yes year, returns sum by year
        #    return(self.data[self.data.year == year]['occurances'].sum())
        #else: # state and year given
        #    return(self.data[(self.data.state == state) & (self.data.year == year)]['occurances'].sum())
        
        
        #
        #if state is None and year is None: # no input, returns general sum
        #    t = self.data
        #elif state is not None and year is None: # no year, yes state, returns sum by state
        #    t = self.data[self.data.state == state]
        #elif state is None and year is not None: # no state, yes year, returns sum by year
        #    t = self.data[self.data.year == year]
        #else: # state and year given
        #    t = self.data[(self.data.state == state) & (self.data.year == year)]